In [32]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from IPython.display import display

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# read data
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

# 单独保存comment_text
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])


In [33]:
# 显示训练和测试数据
# display(train.head(10))
# display(test.head(10))
# all_text[:10]


In [34]:
# word training
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word', # analyze by word
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

# Learn vocabulary and idf from training set
word_vectorizer.fit(all_text)

# Transform documents to document-term matrix.
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)


In [35]:
# train_word_features

In [ ]:
# character traing
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

# fit text 
char_vectorizer.fit(all_text)

# Transform documents to document-term matrix.
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)


In [ ]:
# 将两个稀疏矩阵合成
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])


In [ ]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

# 遍历7个分类标签
for class_name in class_names:
    # 获取每个分类标签列训练数据
    train_target = train[class_name]
    # 生成逻辑回归对象
    classifier = LogisticRegression(C=0.1, solver='sag')
    # 通过ROC-AUC评估模型得分
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    # 保存得分
    scores.append(cv_score)
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    # fit数据
    classifier.fit(train_features, train_target)
    # 预测得分
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

# 保存输出文件
submission.to_csv('submission.csv', index=False)


In [ ]:
# submission

ROC和AUC是评价分类器的指标。ROC是受试者工作特征曲线的简写，又称为感受性曲线。得此名在于曲线上各点反映相同的感受性，它们都是对同一信号刺激的反应，只不过是在几种不同的判定标准下所得的结果而已。ROC纵轴是“真正例率”，横轴是“假正例率”。AUC是处于ROC下方的那部分面积大小。通常，AUC的值介于0.5和1之间，AUC的值越大，诊断准确性越高。在ROC曲线上，靠近坐标图左上方的点为敏感性和特异性均较高的临界值。


In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
#导入数据
filename = '/home/hadoop/DataSet/pima indians.csv'
names = ['preg','plas','pres','skin','test','mass','pedi','age','class']
data = read_csv(filename,names=names)
#将数据分为输入数据和输出结果
array = data.values
X = array[:,0:8]
Y = array[:,8]
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds,random_state=seed)
model = LogisticRegression()
scoring = 'roc_auc'
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
print('AUC %.3f (%.3f)'%(result.mean(),result.std()))